## Full Text Search / Information Retrieval 

## 1) : Why build an Inverted Index for text-retrieval?

## 2) : Objectives:
* Collect a corpus
* Perform a brute-force query and measure the performance
* Create a forward and inverted index from the above 
* Search on one or a combination of terms and measure performance

In [1]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from collections import defaultdict
import time
import re

In [2]:
corpus = fetch_20newsgroups(remove=['header', 'footer'])['data']

In [11]:
corpus[0].lower().split()

['from:',
 'lerxst@wam.umd.edu',
 "(where's",
 'my',
 'thing)',
 'subject:',
 'what',
 'car',
 'is',
 'this!?',
 'nntp-posting-host:',
 'rac3.wam.umd.edu',
 'organization:',
 'university',
 'of',
 'maryland,',
 'college',
 'park',
 'lines:',
 '15',
 'i',
 'was',
 'wondering',
 'if',
 'anyone',
 'out',
 'there',
 'could',
 'enlighten',
 'me',
 'on',
 'this',
 'car',
 'i',
 'saw',
 'the',
 'other',
 'day.',
 'it',
 'was',
 'a',
 '2-door',
 'sports',
 'car,',
 'looked',
 'to',
 'be',
 'from',
 'the',
 'late',
 '60s/',
 'early',
 '70s.',
 'it',
 'was',
 'called',
 'a',
 'bricklin.',
 'the',
 'doors',
 'were',
 'really',
 'small.',
 'in',
 'addition,',
 'the',
 'front',
 'bumper',
 'was',
 'separate',
 'from',
 'the',
 'rest',
 'of',
 'the',
 'body.',
 'this',
 'is',
 'all',
 'i',
 'know.',
 'if',
 'anyone',
 'can',
 'tellme',
 'a',
 'model',
 'name,',
 'engine',
 'specs,',
 'years',
 'of',
 'production,',
 'where',
 'this',
 'car',
 'is',
 'made,',
 'history,',
 'or',
 'whatever',
 'info',

---

### Brute Force

#### Lets start with a example for one term query

In [24]:
def brute_force_query(query, corpus): 
    """Returns indices of matching documents"""
    doc_ids = []
    for i, doc in enumerate(corpus):
        for word in re.findall('(?u)\\b\\w\\w+\\b',doc.lower()):
            if word == query:
                doc_ids.append(i)
    return doc_ids

In [25]:
brute_force_query('bricklin', corpus)

[0, 958, 2835, 4988]

#### Ok lets measure the performance of this approach

In [26]:
%timeit brute_force_query('bricklin', corpus)

2.74 s ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

### Create a forward and an inverted Index 

In [27]:
def create_forward_index(corpus):
    forward_index = defaultdict(list)
    for i, doc in enumerate(corpus):
        forward_index[i] = re.findall('(?u)\\b\\w\\w+\\b',doc.lower()) #recognise this regex anyone?
    return forward_index

In [39]:
def create_inverted_index(corpus):
    index = defaultdict(list) 
    for i, doc in enumerate(corpus):
        for word in re.findall('(?u)\\b\\w\\w+\\b',doc.lower()):
            index[word].append(i)
    return index

#### Ok lets measure the performance of this approach

In [40]:
result = create_inverted_index(corpus)

In [41]:
%timeit create_inverted_index(corpus)

5.1 s ± 240 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Is this what we expected?

In [42]:
%timeit result['bricklin']

94.2 ns ± 6.14 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


---

### Rank the documents which contain the most references

In [45]:
def find_terms(query, index):
    query = query.lower().split()
    results = defaultdict(list) 
    for term in query:
        results[term] = index[term]
    return results

In [52]:
car_results = find_terms('bricklin car', result) 

In [70]:
# How do we find docs where both terms appear?!
#perform a union on the two sets

---

### Other things to consider:

* Results ranking isn't as simple as finding all the results!!
    * e.g - which documents should come back first requires some page ranking - cf Markov and his buddies!!
    * e.g - intra-doc ranking - titles/links could be more important than words in text
    * e.g - intra-doc ranking - long documents will have more occurances of a term
* standard NLP: tokenization, stop words, language, remove crap, parsing, encoding
* typos: Levenshtein distance "hippo" - "hypo" -> Levenshtein dist == 2
* trigram search (heuristic algorithm)

---

### Technologies that utilize inverted-index from scratch:

* Postgres
* Mongodb
* Elasticsearch

---